### Assignment.1
На основе https://en.wikipedia.org/wiki/ISO_4217 и таблицы Active ISO 4217 currency codes создать csv файл, состоящий из столбцов (Code, Currency, Locations listed for the currency)

In [18]:
import os
import requests
from bs4 import BeautifulSoup
import csv

def get_info() -> list:
    """Получаем данне о валютах
    Raises:
        Exception: Если не удалось подключиться к
            https://en.wikipedia.org, поднимается исключение.

    Returns:
        list: Список валют вида [['Code', 'Currency', 'Locations'], ...].
    """

    URL = "https://en.wikipedia.org/wiki/ISO_4217"

    result = []

    # Получаем текст страницы.
    response = requests.get(URL)

    if not response or response.status_code != 200:
        raise Exception(
            f"Не удалось получить данные по адресу {URL}. " +
            f"Ответ от сервера = {response.status_code}")

    soup = BeautifulSoup(response.text, "html.parser")
    for s in soup.select('sup'):
        s.extract()
    
    # Находим таблицы.
    tables = soup.find_all("table")
    for tbl in tables:
        caption = tbl.find("caption")
        if caption and "Active ISO 4217 currency codes" in caption.text:
            table = tbl
            break
    
    # Парсим данные из таблицы.
    rows = table.find("tbody").find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        if not cols:
            continue

        def get_text(cols, colindex, tag = None, tagindex = None):
            if not cols or not cols[colindex]:
                return ""

            result = cols[colindex].text

            if tag != None:
                data = cols[colindex].find_all(tag)
                if tagindex == None:
                    result = ",".join(
                        [value.text.strip("\r\n\t \xa0")
                            for value in data if value])
                elif len(data) > tagindex:
                    result = data[tagindex].text

            return result

        data = []
        data.append(get_text(cols, 0))          # Code
        data.append(get_text(cols, 3, "a", 0))  # Currency
        data.append(get_text(cols, 4, "a"))     # Locations listed for the currency

        result.append(data)

    return result

def write_file(file_name: str, dump: list, columns: list = None) -> None:
        """Запись данных о валютах в csv файл.

        Args:
            file_name (str): Имя файла.
            dump (list): Список строк вида [['строка', ..], ['другая строка', ..]].
            columns(lsit): Список заголовков каждого столбца.
        """
        with open(file_name, "w", encoding="utf-8", newline = "") as file:
            writer = csv.writer(file)
            if columns:
                writer.writerow(columns)
            writer.writerows(dump)


# Записываем в csv файл данные о валютах.
write_file(
    os.path.join("data", "result_file.csv"),
        get_info(), 
        ["Code", "Currency", "Locations listed for the currency"]
    )            



### Assignment.2
Вывести 5 валют, которые используются в наибольшем количестве стран

In [25]:
import pandas as pd

data = pd.read_csv(os.path.join("data", "result_file.csv"))
data["Count"] = data["Locations listed for the currency"].apply(lambda s: str(s).count(',') + 1)
res = data.nlargest(5, "Count")
res
